In [1]:
!pip install -q transformers==4.46.2
!pip install -q qwen-vl-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 72.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 52.1 MB/s eta 0:00:00:00:0100:01


## Download Qwen2-VL-7B-Instruct

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import requests, torch

# 1. Load the model (7B instruct) with FP16 on available GPUs
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    device_map="auto",          # distribute across GPUs automatically
    torch_dtype=torch.float16   # use half precision to save memory
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

2025-12-16 04:47:37.002640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765860457.202677      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765860457.255168      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

## Chat function

In [7]:
def run(messages):
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    
    inputs = inputs.to(model.device)
    
    with torch.no_grad():
        output_ids = model.generate(
            inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    
    response = processor.batch_decode(
        output_ids[:, inputs.shape[-1]:],
        skip_special_tokens=True
    )[0]
    
    print(response)

## Run Qwen

In [6]:
import torch

# Prompt dạng messages (Qwen dùng chat template)
messages = [
    {
        "role": "user",
        "content": "Explain what a transformer model is in simple terms."
    }
]

run(messages)

A transformer model is a type of artificial intelligence model that is used to process and understand natural language text. It was developed by researchers at Google in 2017 and has since become one of the most popular and widely used models for natural language processing tasks.

The basic idea behind a transformer model is to use a series of "transformer layers" to process the input text. Each transformer layer takes the output of the previous layer and applies a series of mathematical operations to it, including attention mechanisms that allow the model to focus on specific parts of the input text that are relevant to the task at hand.

The transformer model is particularly effective at handling long sequences of text, which makes it well-suited for tasks like language translation, text summarization, and question answering. It has also been used to achieve state-of-the-art results on a variety of other natural language processing tasks, such as sentiment analysis and text classifi